In [2]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

2.6.0+cu126
12.6
90501
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 SUPER


In [3]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file inside 01 - Documentation
env_path = Path("01 - Documentation") / "keys.env"

# Load the .env file from that path
load_dotenv(dotenv_path=env_path)

# Now you can access your keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

print(API_KEY[:4] + "****")  # optional: verify it's working


PKD5****


In [4]:
# Create data client
client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Define request
request_params = StockBarsRequest(
    symbol_or_symbols=["AAPL"],
    timeframe=TimeFrame.Day,
    start=datetime(2023, 1, 1),
    end=datetime(2023, 12, 31)
)

# Fetch data
bars = client.get_stock_bars(request_params)
stockData = bars.df
stockData = stockData.reset_index()


print(stockData.head())
print(stockData.columns)
#print(df.index)
print(stockData.shape)


  symbol                 timestamp     open      high     low   close  \
0   AAPL 2023-01-03 05:00:00+00:00  130.280  130.9000  124.17  125.07   
1   AAPL 2023-01-04 05:00:00+00:00  126.890  128.6557  125.08  126.36   
2   AAPL 2023-01-05 05:00:00+00:00  127.130  127.7700  124.76  125.02   
3   AAPL 2023-01-06 05:00:00+00:00  126.010  130.2900  124.89  129.62   
4   AAPL 2023-01-09 05:00:00+00:00  130.465  133.4100  129.89  130.15   

        volume  trade_count        vwap  
0  124289279.0    1021067.0  125.660032  
1   95426133.0     770045.0  126.643011  
2   88344592.0     665463.0  126.006961  
3   96468673.0     709888.0  128.362627  
4   76653608.0     645367.0  131.545593  
Index(['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'trade_count', 'vwap'],
      dtype='object')
(250, 9)


In [5]:
import plotly.express as px

# Plot AAPL closing price over time
fig = px.line(stockData, x='timestamp', y='close', title='AAPL Closing Prices (2023)')
fig.update_layout(xaxis_title='Date', yaxis_title='Close Price (USD)')
fig.show()


Moving Average on the data

In [6]:
# Calculate moving averages
movingAvg10day = stockData['close'].rolling(window=10).mean()
movingAvg20day = stockData['close'].rolling(window=20).mean()
movingAvg5day = stockData['close'].rolling(window=5).mean()

# Create the figure
fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
# Add the 10-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg10day,
                         mode='lines', name='10-Day SMA'))
# Add the 20-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg20day,
                         mode='lines', name='20-Day SMA'))
# Add the 5-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg5day,
                         mode='lines', name='5-Day SMA'))

fig.update_layout(title='AAPL Closing Price with 10-Day Moving Average',
                  xaxis_title='Date', yaxis_title='Price (USD)')

# Show the plot
fig.show()


Auto Regressive Model Fit - scikit

Takes the last 5 trading days and fits a linear regression to the data.


In [12]:
from sklearn.linear_model import LinearRegression

# Extract the 'close' series and drop any NaNs
close_series = stockData['close'].dropna()
close_array = close_series.values

# Lag setup
lag = 5
X, y = [], []
for i in range(lag, len(close_array)):
    X.append(close_array[i-lag:i])
    y.append(close_array[i])

X, y = np.array(X), np.array(y)

# Fit model
model = LinearRegression().fit(X, y)
ARpreds = model.predict(X)

fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
# Add the AR predictions
fig.add_trace(go.Scatter(x=stockData['timestamp'][lag:], y=ARpreds,
                         mode='lines', name='AR Predictions'))
fig.update_layout(title='AAPL Closing Prices vs AR(5) Model (Scikit-Learn)',
                  xaxis_title='Date', yaxis_title='Price (USD)')
fig.show()